In [3]:
from collections import Counter
import numpy as np
from itertools import product
from typing import List, Dict, Set

### Helpers

In [4]:
def HammingDistance(s1: str, s2: str) -> int:
    ''' Takes in two k-mers and returns Hamming distance between them'''

    res = 0
    for i in range(len(s1)):
        if s1[i] != s2[i]:
            res += 1
    return res

In [5]:
def Neighbors(pattern: str, d: int) -> Set[str]:
    ''' Takes in a k-mer and returns all k-mers that have at most d mismatches with the original'''
    
    if d == 0:
        return {pattern}
    if len(pattern) == 1: 
        return {'A', 'C', 'G', 'T'}
    
    neighborhood = set()
    
    SuffixNeighbors = Neighbors(pattern[1:], d)
    for s in SuffixNeighbors:
        if HammingDistance(pattern[1:], s) < d:
            for nucleotide in {'A', 'C', 'G', 'T'}:
                neighborhood.add(nucleotide + s)
        else:
            neighborhood.add(pattern[0] + s)
    return neighborhood

### Motif Enumeration

In [6]:
def MotifEnumeration(dna: str, k: int, d: int) -> Set[str]:
    ''' Takes in a DNA sequence and returns all (k, d)-motifs found by Brute Force
        where (k,d)-motif is a k-mer that appears in every string from DNA with at most d mismatches 
    
    Parameters:
        dna: DNA string, space separated
          k: lenght of k-mer
          d: max number of allowed mismatches
          
    Returns:
        All (k, d)-motifs in DNA
    '''

    # preprocess input data            
    dna = dna.split(" ")
    dna = [list(row) for row in dna]
    
    patterns_by_row = [set() for _ in dna] # Creating a list of sets length len(seq)
    for row_index, row in enumerate(dna):
        row_set = set()
        for i in range(len(row) - k + 1):
            cur_pattern = row[i:i + k]
            pattern_neighbors = Neighbors(cur_pattern, d)
            patterns_by_row[row_index] = patterns_by_row[row_index].union(pattern_neighbors)           

    patterns = patterns_by_row[0]
    for patterns_set in patterns_by_row:
        patterns &= patterns_set

    return patterns

In [7]:
dna_1 = "ATTTGGC TGCCTTA CGGTATC GAAAATT"

ans = MotifEnumeration(dna_1, k=3, d=1)
print(" ".join(ans))

# right answer: TTT ATT ATA GTT

ATA TTT GTT ATT


In [8]:
dna_2 = "CGCACCTATCATAGCTTGCTGATGT CGGGTCGCAAGATAAACAATTCCCG CGCACCGGTCCATAGGCCAGGATGA GTTGCCGCATACACCTTTAGTTTGG GAGGTTATATCGCATCGAGAAACGG ATCCTGAACGCCCTTCGCATGAAAG"

ans = MotifEnumeration(dna_2, k=5, d=1)
print(" ".join(ans))

# right answer: CGCAT CGCAC CGGAT CGCAA CGCAG GCATG

CGCAC CGCAT CGCAG CGCAA GCATG CGGAT


### Median String

In [9]:
def DistanceBetweenPatternAndStrings(pattern: str, dna: str) -> float:
    ''' Takes in a DNA sequence and a pattern and returns distance between them
    
    Parameters:
        pattern: pattern in str format
            dna: DNA string, space separated
          
    Returns:
        distance: distance between DNA and pattern
    '''

    dna = dna.split(" ")

    k = len(pattern)
    distance = 0
    for string in dna:
        hamming_distance = float('inf')
        # check all k-mers in the current string
        for i in range(len(string) - k + 1):
            k_mer = string[i:i + k]
            d = HammingDistance(pattern, k_mer)
            hamming_distance = min(hamming_distance, d)
            
        distance += hamming_distance
        
    return distance

In [10]:
dna = "TTACCTTAAC GATATCTGTC ACGGCGTTCG CCCTAAAGAG CGTCAGAGGT"

DistanceBetweenPatternAndStrings("AAA", dna)

5

In [11]:
def MedianString(dna: str, k: int) -> List[str]:
    ''' Takes in a DNA sequence and an integer k 
        and returns a k-mer Pattern(s) that minimizes d(Pattern, DNA)
        among all possible choices of k-mers
    
    Parameters:
            dna: DNA string, space separated
              k: k-mer length
          
    Returns:
        median: a list of k-mer Patterns
    '''    
    
    distance = float('inf')
    nucleotides = ["A", "T", "C", "G"]
    median = []
    # create all possible pattern with length k
    prod = [p for p in product(nucleotides, repeat=k)]
    possible_patterns = ["".join(item) for item in prod]
    
    for pattern in possible_patterns:
        d = DistanceBetweenPatternAndStrings(pattern, dna)

        if distance > d:
            distance = d
            median = [pattern]
        elif distance == d:
            median.append(pattern)
            
    return median

In [12]:
dna = "AAATTGACGCAT GACGACCACGTT CGTCAGCGCCTG GCTGAGCACCGG AGTTCGGGACAG"
k = 3

ans = MedianString(dna, k)
print(" ".join(ans))
# right answer: GAC

GAC


In [13]:
dna = "CTCGATGAGTAGGAAAGTAGTTTCACTGGGCGAACCACCCCGGCGCTAATCCTAGTGCCC GCAATCCTACCCGAGGCCACATATCAGTAGGAACTAGAACCACCACGGGTGGCTAGTTTC GGTGTTGAACCACGGGGTTAGTTTCATCTATTGTAGGAATCGGCTTCAAATCCTACACAG"

ans = MedianString(dna, 7)
print(" ".join(ans))
# right answer: AATCCTA TAGTTTC GAACCAC GTAGGAA

AATCCTA TAGTTTC GAACCAC GTAGGAA


### Greedy Motif Search

In [14]:
def get_score(motifs: List[List[str]]) -> float:
    '''
    Returns motifs' score
    
        Parameters:
            motifs: motifs array, every line is a DNA string, every cell contains a nucleotide
                    e.g. [['G', 'G', 'C'],
                          ['A', 'A', 'G']]
        Returns:
            score: motif score
    '''
    
    # convert input array into numpy array to simplify the work with columns
    motifs = np.array(motifs)
    n, m = len(motifs), len(motifs[0])
    
    score = 0
    
    for col in range(m):
        # calculate nucleotides counter for one column
        counter = Counter(motifs[:, col])
        max_value = max(counter.values())
        score += n - max_value
        
    return score

In [15]:
motifs = [['G', 'G', 'C'],
          ['A', 'A', 'G'],
          ['C', 'A', 'A'],
          ['C', 'A', 'C'],
          ['C', 'A', 'A']]

get_score(motifs)

6

In [16]:
def get_profile(dna: List[List[str]]) -> Dict[str, float]:
    ''' Takes in a DNA sequence and returns its profile
    
    Parameters:
            dna: a DNA sequence in list format
            e.g. [['G', 'G', 'C'], ['A', 'A', 'G']]
          
    Returns:
        profile: a dictionary with nucleotides A, C, G, T as keys 
                 and probabilities as values
    '''        
    
    dna = np.array(dna)
    n, m = len(dna), len(dna[0])
    
    profile = {
        "A": [0]*m,
        "C": [0]*m,
        "G": [0]*m,
        "T": [0]*m
    }
    
    for col in range(m):
        counter = Counter(dna[:, col])
        
        base = sum(counter.values())
        
        for item, value in counter.items():
            profile[item][col] = value / base
            
    return profile    

In [22]:
ans = get_profile([['G', 'G', 'C'], ['A', 'A', 'G']])
print(ans)

{'A': [0.5, 0.5, 0], 'C': [0, 0, 0.5], 'G': [0.5, 0.5, 0.5], 'T': [0, 0, 0]}


In [17]:
def get_profile_laplace(motifs: List[List[str]]) -> Dict[str, List[float]]:
    '''         
        Parameters:
            motifs: motifs array, every line is a DNA string, every cell contains a nucleotide
                    e.g. [['G', 'G', 'C'],
                          ['A', 'A', 'G']]
        Returns:
            profile: motifs' profile in dictionary format,
                     incorporating Laplace's Rule of Succession (no 0 probabilities in the answer)
                     e.g. {'A': [0.33, 0.33, 0.16],
                           'C': [0.16, 0.16, 0.33],
                           'G': [0.33, 0.33, 0.33],
                           'T': [0.16, 0.16, 0.16]}
    '''
    
    motifs = np.array(motifs)
    n, m = len(motifs), len(motifs[0])
    
    profile = {
        "A": [1] * m,
        "C": [1] * m,
        "G": [1] * m,
        "T": [1] * m
    }
    
    for col in range(m):
        counter = Counter(motifs[:, col])        
        base = sum(counter.values()) + 4 # we add 1 to each of 4 nucleotides
        
        for nucleotide, arr in profile.items():
            if nucleotide in counter:
                profile[nucleotide][col] += counter[nucleotide]
                profile[nucleotide][col] /= base
            else:
                profile[nucleotide][col] /= base
            
    return profile

In [18]:
ans = get_profile_laplace([['G', 'G', 'C'], ['A', 'A', 'G']])
print(ans)

{'A': [0.3333333333333333, 0.3333333333333333, 0.16666666666666666], 'C': [0.16666666666666666, 0.16666666666666666, 0.3333333333333333], 'G': [0.3333333333333333, 0.3333333333333333, 0.3333333333333333], 'T': [0.16666666666666666, 0.16666666666666666, 0.16666666666666666]}


In [30]:
def find_most_prob_kmer(dna: str, profile: Dict[str, float], k: int) -> str:
    ''' Takes in a DNA sequence, its profile 
        and returns the most probable k-mer based on it
    
    Parameters:
                dna: DNA string, space separated
            profile: DNA's profile
          
    Returns:
        best_k_mer: the most probable k-mer in the DNA string
    '''     
    
    n = len(dna)
    best_k_mer = None
    max_prob = float('-inf')

    for i in range(n - k + 1):
        k_mer = dna[i:i + k]

        prob = 1
        for index, nucleotide in enumerate(k_mer):
            prob *= profile[nucleotide][index]

        if prob > max_prob:
            max_prob = prob
            best_k_mer = k_mer

    return best_k_mer   

In [31]:
profile = {
"A": [0, 0, 0],
"C": [0, 0, 1],
"G": [1, 1, 0],
"T": [0, 0, 0]
}

ans = find_most_prob_kmer("AAGAATCAGTCA", profile, 3)
print(ans)

AAG


In [50]:
def GreedyMotifSearch(dna: str, k: int, t: int) -> List[str]:
    ''' Takes in a DNA sequence and returns best motifs
    
    Parameters:
                dna: DNA string, space separated
                  k: k-mer length
                  t: number of DNA lines to use
          
    Returns:
        motifs: a collection of strings representing best motifs
    '''      
    
    
    dna = dna.split(" ")
    dna = [list(row) for row in dna]
    
    n, m = len(dna), len(dna[0])
    
    best_motifs = None
    best_score = float("inf")
    
    for i in range(m - k + 1):
        start_kmer = dna[0][i:i + k]
        
        motifs = [start_kmer]

        for row in range(1, t):
            profile = get_profile_laplace(motifs)
            
            best_kmer = list(find_most_prob_kmer("".join(dna[row]), profile, k))
            motifs.append(best_kmer)            
            
        motifs_score = get_score(motifs)
        
        if motifs_score < best_score:
            best_score = motifs_score
            best_motifs = motifs
            
    return ["".join(row) for row in best_motifs]    

In [51]:
dna = "GGCGTTCAGGCA AAGAATCAGTCA CAAGGAGTTCGC CACGTCAATCAC CAATAATATTCG"
ans = GreedyMotifSearch(dna, 3, 5)
print(" ".join(ans))

TTC ATC TTC ATC TTC
